
# H2O Solution for ML challenge:
https://www.hackerearth.com/challenge/competitive/machine-learning-challenge-one/

original dataset:
https://www.hackerearth.com/problem/machine-learning/bank-fears-loanliness/

This file uses the .csv file of transformed data output by file `RF_loan_default_prob.ipynb`

In [24]:
import h2o
import os
import numpy as np
from sklearn import metrics
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [25]:
h2o.init(max_mem_size = "2G")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (build 1.8.0_121-8u121-b13-0ubuntu1.16.10.2-b13); OpenJDK 64-Bit Server VM (build 25.121-b13, mixed mode)
  Starting server from /media/badicst/875664ae-1937-4e8e-a7f3-43b9b3e28b21/anaconda3/envs/ipykernel_py2/h2o_jar/h2o.jar
  Ice root: /tmp/tmp_7VHuJ
  JVM stdout: /tmp/tmp_7VHuJ/h2o_badicst_started_from_python.out
  JVM stderr: /tmp/tmp_7VHuJ/h2o_badicst_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,5 months and 5 days !!!
H2O cluster name:,H2O_from_python_badicst_20h200
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [26]:
h2o.remove_all()

In [27]:
train_df = h2o.import_file(os.path.realpath('data_transformed.csv'))
test_df = h2o.import_file(os.path.realpath('test_transformed.csv'))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
trainl, validl, testl = train_df.split_frame([0.6, 0.2], seed=1234)

In [30]:
#help(H2ORandomForestEstimator)

In [31]:
from enum import Enum
rf_v1 = H2ORandomForestEstimator(
    model_id="rf_covType_v1",
    balance_classes=True,
    max_depth=50,
    ntrees=1000,
    stopping_rounds=2,
    stopping_tolerance=0.0001,
    score_each_iteration=True,
    seed=1000000)

In [32]:

X = trainl.col_names[:-1]     #last column is the response variable 
y = trainl.col_names[-1] 

In [33]:
#help(rf_v1.train)

# Training the model

In [34]:
rf_v1.train(X, y, training_frame=trainl, validation_frame=validl)

drf Model Build progress: |███████████████████████████████████████████████| 100%


# Prediction

In [35]:
prediction_default  = rf_v1.predict(test_df)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [36]:
prediction_default=prediction_default+0.0000001

submitfile_h2o=test_df[['member_id']]

submitfile_h2o['loan_status']=prediction_default

submitfile_h2o=submitfile_h2o.as_data_frame(True)

submitfile_h2o["loan_status"]=submitfile_h2o["loan_status"]/1.1

In [37]:
submitfile_h2o.to_csv("submissionh2o2.csv", index=False)

In [38]:
#help(rf_v1.predict)

In [39]:
rf_v1.score_history()

/media/badicst/875664ae-1937-4e8e-a7f3-43b9b3e28b21/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: `score_history` is deprecated. Use `scoring_history`
  if __name__ == '__main__':


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2017-04-09 13:28:00,0.055 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2017-04-09 13:28:12,12.326 sec,1.0,0.387177,0.149933,0.149906,0.391724,0.153472,0.153447
2,,2017-04-09 13:28:22,21.834 sec,2.0,0.376559,0.151342,0.141797,0.332599,0.153043,0.110622
3,,2017-04-09 13:28:31,31.324 sec,3.0,0.367060,0.153228,0.134733,0.310878,0.153309,0.096645
4,,2017-04-09 13:28:41,41.233 sec,4.0,0.358666,0.155157,0.128641,0.300634,0.155362,0.090381
5,,2017-04-09 13:28:52,51.959 sec,5.0,0.349999,0.156580,0.122499,0.294997,0.156953,0.087023
6,,2017-04-09 13:29:01,1 min 1.578 sec,6.0,0.341671,0.157839,0.116739,0.290715,0.158397,0.084515
7,,2017-04-09 13:29:11,1 min 10.881 sec,7.0,0.333015,0.157491,0.110899,0.286911,0.158101,0.082318
8,,2017-04-09 13:29:20,1 min 20.172 sec,8.0,0.326409,0.158027,0.106543,0.284035,0.158251,0.080676
9,,2017-04-09 13:29:30,1 min 30.281 sec,9.0,0.319233,0.157054,0.101910,0.281119,0.157446,0.079028


# Computing score on test set using sklearn:

In [40]:
final_rf_predictions = rf_v1.predict(testl[:-1])
#test set accuracy


drf prediction progress: |████████████████████████████████████████████████| 100%


In [41]:
pred=final_rf_predictions.as_data_frame(use_pandas=True)
pred=np.array(pred["predict"])
true_labels=np.array(testl["loan_status"].as_data_frame(use_pandas=True))
fpr, tpr, thresholds = metrics.roc_curve(true_labels, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.94124005834774238

In [42]:

#hackerearth:0.93624160393457545 for median 0.93773214347128286 with -1 
#AV: 0.94124005834774238

In [43]:
my_rf_metrics = rf_v1.model_performance(validl)
my_rf_metrics.show()


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.0717389349068
RMSE: 0.26784124945
MAE: 0.159747166214
RMSLE: 0.187654493866
Mean Residual Deviance: 0.0717389349068


In [44]:
h2o.shutdown(prompt=False)

H2O session _sid_b9b5 closed.
